In [1]:
import numpy as np
import pandas as pd
import cv2 
import pytesseract
from glob import glob
import spacy
import os
import string

In [2]:


def cleanText(txt):
    whitespace = string.whitespace
    punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'
    tabeWhiteSpace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text=str(txt)
    text = text.lower()
    removeWhiteSpace = text.translate(tabeWhiteSpace)
    removePunctuation = removeWhiteSpace.translate(tablePunctuation)
    return str(removePunctuation)    

In [4]:
#Load NER model
model_ner = spacy.load('./output/model-best/')

In [82]:
# Load Image
image = cv2.imread('./Selected/N009.jpg')
# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)
#tessData
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

home master hardware electrical nq.113g 1156, jalan setia gembilang u13/gg bandar setia alam, 40770 bandar setia alam, selangor, company reg na. s403711951-f gst reg no. 001837511166 tak invoice invole no. «cs 87832 date 22/12/2017 1403 cashler cashier rm code 24nimw7y m.one tape 1.00 ¥ 15.90 15.90 sh subtotal 75.90 totalexclofgst 45.00 tota incl. of gst 15.90 totalamtrounded 45.90 payment 50.00 change due 34.10 - total tema 4 sg ummary amounirm tax rm sr @a 18.00 o.90 sissncees thane yyy saeeenees ary goods return please 00 within ¢ cays with original receipt ta”


In [84]:
from spacy import displacy

In [38]:
#displacy.serve(doc,style='ent')

In [85]:
displacy.render(doc,style='ent')

### Tagging

In [86]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [87]:
doc_text = docjson['text']
doc_text

'home master hardware electrical nq.113g 1156, jalan setia gembilang u13/gg bandar setia alam, 40770 bandar setia alam, selangor, company reg na. s403711951-f gst reg no. 001837511166 tak invoice invole no. «cs 87832 date 22/12/2017 1403 cashler cashier rm code 24nimw7y m.one tape 1.00 ¥ 15.90 15.90 sh subtotal 75.90 totalexclofgst 45.00 tota incl. of gst 15.90 totalamtrounded 45.90 payment 50.00 change due 34.10 - total tema 4 sg ummary amounirm tax rm sr @a 18.00 o.90 sissncees thane yyy saeeenees ary goods return please 00 within ¢ cays with original receipt ta”'

In [88]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,home
1,1,5,11,master
2,2,12,20,hardware
3,3,21,31,electrical
4,4,32,38,nq.113
5,5,38,39,g
6,6,40,44,1156
7,7,44,45,","
8,8,46,51,jalan
9,9,52,57,setia


In [90]:
doc_text[21:31]

'electrical'

In [91]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [92]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,home,B-ORG
1,1,5,11,master,I-ORG
2,2,12,20,hardware,I-ORG
3,3,21,31,electrical,I-ORG
4,4,32,38,nq.113,O
5,5,38,39,g,O
6,6,40,44,1156,O
7,7,44,45,",",O
8,8,46,51,jalan,O
9,9,52,57,setia,O


In [93]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\1286819257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1
C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\1286819257.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)


In [94]:
# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [95]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
82,5,1,6,1,2,3,233,1260,106,22,94,return,520,514,return,O
83,5,1,6,1,2,4,347,1261,98,22,95,please,527,521,please,O
84,5,1,6,1,2,5,454,1263,37,21,76,00,530,528,00,O
85,5,1,6,1,2,6,499,1263,94,22,95,within,537,531,within,O
86,5,1,6,1,3,1,94,1297,12,20,48,¢,539,538,¢,O
87,5,1,6,1,3,2,116,1297,68,22,86,cays,544,540,cays,O
88,5,1,6,1,3,3,190,1298,64,23,94,with,549,545,with,O
89,5,1,6,1,3,4,263,1301,120,22,94,original,558,550,original,O
90,5,1,6,1,3,5,392,1302,113,22,87,receipt,566,559,receipt,O
91,5,1,6,1,3,6,512,1303,58,22,34,ta”,570,567,ta,O


### Bounding Box

In [96]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [97]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\2854881843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,1,1,1,1,151,50,84,26,95,home,4,0,home,ORG
1,5,1,1,1,1,2,243,51,115,29,96,master,11,5,master,ORG
2,5,1,1,1,1,3,368,57,166,26,93,hardware,20,12,hardware,ORG
3,5,1,2,1,1,1,268,91,174,31,95,electrical,31,21,electrical,ORG
33,5,1,4,1,3,2,250,478,127,22,75,22/12/2017,231,221,22/12/2017,DATE


In [98]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [99]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\472836701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


In [100]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
0,5,1,1,1,1,1,151,50,84,26,95,home,4,0,home,ORG,1
1,5,1,1,1,1,2,243,51,115,29,96,master,11,5,master,ORG,1
2,5,1,1,1,1,3,368,57,166,26,93,hardware,20,12,hardware,ORG,1
3,5,1,2,1,1,1,268,91,174,31,95,electrical,31,21,electrical,ORG,1
33,5,1,4,1,3,2,250,478,127,22,75,22/12/2017,231,221,22/12/2017,DATE,2
55,5,1,4,1,10,5,448,793,62,21,86,15.90,362,357,15.90,MONEY,3


In [101]:
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\262737354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\262737354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] + bb_df['width']
C:\Users\oussemanaffety\AppData\Local\Temp\ipykernel_8084\262737354.py:4: SettingWithCopyWarning: 
A value is trying to be set on a 

In [102]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [103]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [104]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,151,534,50,122,[ORG],home master hardware electrical
2,250,377,478,500,[DATE],22/12/2017
3,448,510,793,814,[MONEY],15.90


In [105]:
img_bb = image.copy()
for l, r, t, b, label, token in img_tagging.values:
    cv2.rectangle(img_bb, (l, t), (r, b), (0, 255, 0), 2)
    
    # Convert label to a string if it's not already
    label_str = str(label)
    cv2.putText(img_bb, label_str, (l, t), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255), 2)

cv2.imshow('Bounding Box Invoice', img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Parser

In [106]:
import re

def parser(text, label):
    if label == 'DATE':
        text = text.lower()

        
    elif label == 'CARDINAL':
        text = re.sub(r'[^0-9]', '', text)
        
    elif label == 'MONEY':
        text = text.lower()
        text = re.sub(r'[^0-9$€£,.]', '', text)
        
    elif label == 'ORG':
        text = text.title()
        
    return text


#### Entities

In [107]:
import re

In [108]:
info_array = dataframe_info[['token','label']].values
entities = dict(ORG=[],DATE=[],MONEY=[],CARDINAL=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    # step -1 parse the token
    text = parser(token,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("ORG",'DATE','MONEY','CARDINAL'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
                    
    
    
    previous = label_tag


In [109]:
entities

{'ORG': ['Home Master Hardware Electrical'],
 'DATE': ['22/12/2017'],
 'MONEY': ['15.90'],
 'CARDINAL': []}